## WAR 타자

In [6]:
import pandas as pd
for y in range(2000,2020):

    df = pd.read_csv('./war/WAR{}.csv'.format(y))[['PLAYER','TEAM','AVG',	'OBP',	'SLG',	'OPS',	'WAR']]
  # 데이터가 없는 컬럼명이 들어있는 row제거
    df = df[df.WAR.apply(lambda x: str(x)) != 'WAR']
  # string->float
    df['bWAR'] = df.WAR.apply(lambda x: float(x))
    df['OPS'] = df.OPS.apply(lambda x: float(x))
    df['SLG'] = df.SLG.apply(lambda x: float(x))
    df['AVG'] = df.AVG.apply(lambda x: float(x))
    df['OBP'] = df.OBP.apply(lambda x: float(x))
    df.drop('WAR',axis=1)
    # WAR 0이상
    #df = df[df.WAR.apply(lambda x: float(x))>=0]
    # 팀히스토리 컬럼 그 시즌에 거친 팀의 리스트 
    df['TEAMHISTORY'] = df.TEAM.apply(lambda x: str(x).split('/'))

    # 그해 있었던 팀 리스트 만들기
    teamlist = [str(i) for i in df.TEAM.unique() if '/' not in str(i)]
    csv = pd.DataFrame()
    for t in teamlist:
        foo = pd.DataFrame(df[df.TEAMHISTORY.apply(lambda x: t in x)].mean(), columns=[t])
        csv = pd.concat([csv, foo], axis=1)
        
    csv.to_csv('./war/WAR_TEAM_AVG{}.csv'.format(y))
    print(y)

2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019


## WAR 투수

In [7]:
import pandas as pd
for y in range(2000,2020):
    foo = pd.DataFrame()
    df = pd.read_csv('./war/WAR{}_p.csv'.format(y))[['PLAYER','TEAM','WAR','WHIP','ERA']]
    # 데이터가 없는 컬럼명이 들어있는 row제거
    df = df[df.WAR.apply(lambda x: str(x)) != 'WAR']
    # string->float
    df['pWAR'] = df.WAR.apply(lambda x: float(x))
    df['WHIP'] = df.WHIP.apply(lambda x: float(x))
    df['ERA'] = df.ERA.apply(lambda x: float(x))
    df.drop('WAR',axis=1)
    # WAR 0이상
    #df = df[df.WAR.apply(lambda x: float(x))>=0]
    # 팀히스토리 컬럼 그 시즌에 거친 팀의 리스트 
    df['TEAMHISTORY'] = df.TEAM.apply(lambda x: str(x).split('/'))

    # 그해 있었던 팀 리스트 만들기
    teamlist = [str(i) for i in df.TEAM.unique() if '/' not in str(i)]
    csv = pd.DataFrame()
    for t in teamlist:
        foo = pd.DataFrame(df[df.TEAMHISTORY.apply(lambda x: t in x)].mean(), columns=[t])
        csv = pd.concat([csv, foo], axis=1)
        
    csv.to_csv('./war/WAR_TEAM_AVG{}_p.csv'.format(y))
    print(y)

2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019


## 스코어

In [8]:
df = pd.read_csv('MLB_scores.csv')
df['RESULT'] = df.RESULT.apply(lambda x: str(x).split('(')[0])
df = df[df.RESULT.str.strip() != 'LIVE']
df = df[df.RESULT.str.strip() != 'Postponed']
df = df[df.RESULT.str.strip() != 'Canceled']
df = df[df.RESULT.apply(lambda x: '-' not in str(x))]
df = df[df.RESULT.apply(lambda x: 'GAME' not in str(x))]
df = df[df.RESULT != 'nan']

df['RESULTLIST'] = df.RESULT.apply(lambda x: str(x).split(','))
df['RESULTLIST'] = df.RESULTLIST.apply(
    lambda x: [ 
        tuple( x[0].split()),
        tuple( x[-1].split())  
    ]
)
df['YEAR'] = df.DATE.apply(lambda x: str(x)[0:4])
df['HOME'] = df.HOME.apply(lambda x: x.split()[-1].strip())
df['AWAY'] = df.AWAY.apply(lambda x: x.split()[-1].strip())
df['winner'] = df.RESULTLIST.apply(lambda x: x[0][0])

df_home = df[df.winner == df.HOME]
df_away = df[df.winner == df.AWAY]

df_home['SCORE'] = df.RESULTLIST.apply(lambda x: int(x[0][-1]) - int(x[-1][-1]))
df_home = df_home[['HOME', 'AWAY', 'YEAR', 'SCORE']]
df_away['SCORE'] = df.RESULTLIST.apply(lambda x: int(x[-1][-1]) - int(x[0][-1]))
df_away = df_away[['HOME', 'AWAY', 'YEAR', 'SCORE']]

csv = pd.concat([df_home, df_away])
csv.to_csv('MLB_scores_proc.csv')

C:\Users\103-04\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\103-04\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## 스코어와 WAR 조인

In [9]:
foo = pd.DataFrame()
for y in range(2000,2020):
    print(y)
  
    score = pd.read_csv('MLB_scores_proc.csv')
    score[score.YEAR == y+1]
    score = pd.read_csv('MLB_scores_proc.csv')
    score = score[score.YEAR == y-1]
    batter_war = pd.read_csv('./war/WAR_TEAM_AVG{}.csv'.format(y)).T
    pitcher_war = pd.read_csv('./war/WAR_TEAM_AVG{}_p.csv'.format(y)).T
    batter_war.columns = batter_war.iloc[0]
    batter_war = batter_war.iloc[1:]
    batter_war['TEAM'] = batter_war.index
    pitcher_war.columns = pitcher_war.iloc[0]
    pitcher_war = pitcher_war.iloc[1:]
    pitcher_war['TEAM'] = pitcher_war.index

    df = pd.merge(score, batter_war, left_on='HOME', right_on='TEAM')
    df = pd.merge(df,batter_war, left_on='AWAY', right_on='TEAM')
    df = pd.merge(df, pitcher_war, left_on='HOME', right_on='TEAM')
    df = pd.merge(df,pitcher_war, left_on='AWAY', right_on='TEAM')
    foo = pd.concat([foo, df])
foo.to_csv('data.csv')

  

2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
